In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import io
from google.colab import files

In [2]:
uploaded = files.upload()

Saving justice.csv to justice (1).csv


In [3]:
df = pd.read_csv(io.BytesIO(uploaded["justice.csv"]))

In [4]:
df.head()

,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area
0,0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",501,7,2,True,majority opinion,reversed,NaN
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3303 entries, 0 to 3302
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          3303 non-null   int64 
 1   ID                  3303 non-null   int64 
 2   name                3303 non-null   object
 3   href                3303 non-null   object
 4   docket              3303 non-null   object
 5   term                3303 non-null   object
 6   first_party         3302 non-null   object
 7   second_party        3302 non-null   object
 8   facts               3303 non-null   object
 9   facts_len           3303 non-null   int64 
 10  majority_vote       3303 non-null   int64 
 11  minority_vote       3303 non-null   int64 
 12  first_party_winner  3288 non-null   object
 13  decision_type       3296 non-null   object
 14  disposition         3231 non-null   object
 15  issue_area          3161 non-null   object
dtypes: int64(5), object(11)


In [6]:
df_new = df[['facts', 'disposition', 'decision_type', 'issue_area', 'first_party_winner']]

In [7]:
df_new.head()

,facts,disposition,decision_type,issue_area,first_party_winner
0,"<p>In 1970, Jane Roe (a fictional name used in...",reversed,majority opinion,NaN,True
1,<p>Joan Stanley had three children with Peter ...,reversed/remanded,majority opinion,Civil Rights,True
2,<p>John Giglio was convicted of passing forged...,reversed/remanded,majority opinion,Due Process,True
3,"<p>The Idaho Probate Code specified that ""male...",reversed/remanded,majority opinion,Civil Rights,True
4,"<p>Miller, after conducting a mass mailing cam...",vacated/remanded,majority opinion,First Amendment,True


In [8]:
df_new = df_new.dropna()

In [9]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3099 entries, 1 to 3302
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   facts               3099 non-null   object
 1   disposition         3099 non-null   object
 2   decision_type       3099 non-null   object
 3   issue_area          3099 non-null   object
 4   first_party_winner  3099 non-null   object
dtypes: object(5)
memory usage: 145.3+ KB


In [11]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [20]:
# def remove_stopwords(row):
#     sentence = row["facts"]
#     modified_sentence = []
#     sentence = re.sub(r"<[a-z]+>", "", sentence)
#     sentence = re.sub(r"\n", "", sentence)
#     sentence = [sentence.replace(punct, "") for punct in string.punctuation]
#     modified_sentence = [word for word in sentence.split() if word not in stop_words]
#     return " ".join(modified_sentence).lower()

def remove_stopwords(row):
  sentence = row['facts']
  cleaned = []
  sentence = re.sub(r'<[a-z]+>', '', sentence)
  sentence = re.sub(r'\n', '', sentence)
  for punct in string.punctuation:
      sentence = sentence.replace(punct, '')
  for word in sentence.split():
      if word not in stop_words:
          cleaned.append(word)
  return ' '.join(cleaned).lower()

In [21]:
df_new["modified_facts"] = df_new.apply(remove_stopwords, axis = 1)

In [22]:
df_new[["facts", "modified_facts"]]

,facts,modified_facts
1,<p>Joan Stanley had three children with Peter ...,joan stanley three children peter stanley the ...
2,<p>John Giglio was convicted of passing forged...,john giglio convicted passing forged money ord...
3,"<p>The Idaho Probate Code specified that ""male...",the idaho probate code specified males must pr...
4,"<p>Miller, after conducting a mass mailing cam...",miller conducting mass mailing campaign advert...
5,<p>Ernest E. Mandel was a Belgian professional...,ernest e mandel belgian professional journalis...
...,...,...
3297,<p>For over a century after the Alaska Purchas...,for century alaska purchase 1867 federal gover...
3298,"<p>Refugio Palomar-Santiago, a Mexican nationa...",refugio palomarsantiago mexican national grant...
3299,<p>Tarahrick Terry pleaded guilty to one count...,tarahrick terry pleaded guilty one count posse...
3300,<p>Joshua James Cooley was parked in his picku...,joshua james cooley parked pickup truck side r...


In [23]:
print(df_new["facts"].iloc[0])
print('----')
print(df_new["modified_facts"].iloc[0])

<p>Joan Stanley had three children with Peter Stanley.  The Stanleys never married, but lived together off and on for 18 years.  When Joan died, the State of Illinois took the children.  Under Illinois law, unwed fathers were presumed unfit parents regardless of their actual fitness and their children became wards of the state.  Peter appealed the decision, arguing that the Illinois law violated the Equal Protection Clause of the Fourteenth Amendment because unwed mothers were not deprived of their children without a showing that they were actually unfit parents.  The Illinois Supreme Court rejected Stanley’s Equal Protection claim, holding that his actual fitness as a parent was irrelevant because he and the children’s mother were unmarried.</p>

----
joan stanley three children peter stanley the stanleys never married lived together 18 years when joan died state illinois took children under illinois law unwed fathers presumed unfit parents regardless actual fitness children became wa

In [15]:
df_new

,facts,disposition,decision_type,issue_area,first_party_winner
1,<p>Joan Stanley had three children with Peter ...,reversed/remanded,majority opinion,Civil Rights,True
2,<p>John Giglio was convicted of passing forged...,reversed/remanded,majority opinion,Due Process,True
3,"<p>The Idaho Probate Code specified that ""male...",reversed/remanded,majority opinion,Civil Rights,True
4,"<p>Miller, after conducting a mass mailing cam...",vacated/remanded,majority opinion,First Amendment,True
5,<p>Ernest E. Mandel was a Belgian professional...,reversed,majority opinion,First Amendment,True
...,...,...,...,...,...
3297,<p>For over a century after the Alaska Purchas...,reversed/remanded,majority opinion,Civil Rights,True
3298,"<p>Refugio Palomar-Santiago, a Mexican nationa...",reversed/remanded,majority opinion,Criminal Procedure,True
3299,<p>Tarahrick Terry pleaded guilty to one count...,affirmed,majority opinion,Criminal Procedure,False
3300,<p>Joshua James Cooley was parked in his picku...,vacated/remanded,majority opinion,Civil Rights,True
